<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Asset Management

### Larger Data Set

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

## Index Constituents

Topics of interest include:

* API Connection to Refinitiv Eikon (not required)
* Retrieving end-of-day time series data for a complete index
* Retrieving market capitalization data for the index constituents

## Real Data

**_Historical end-of-day financial time series & market cap data._**

Data retrieved vom Refinitiv Eikon for the German DAX 30 constituents.

The data sets:

    http://hilpisch.com/dax_eikon_eod_data.csv
    http://hilpisch.com/dax_eikon_mc_data.csv

## Imports and Data

In [ ]:
!git clone https://github.com/tpq-classes/python_for_asset_management.git
import sys
sys.path.append('python_for_asset_management')


In [ ]:
!pip install eikon

In [ ]:
import math
import eikon as ek
import numpy as np
import pandas as pd
from pylab import plt
import configparser as cp
plt.style.use('seaborn-v0_8')
pd.set_option('display.precision', 4)
np.set_printoptions(suppress=True, precision=4)
%config InlineBackend.figure_format = 'svg'

## API Connection

In [ ]:
c = cp.ConfigParser()
c.read('../eikon.cfg')

In [ ]:
ek.set_app_key(c['eikon']['app_key'])

## Time Series Data

In [ ]:
data = ek.get_timeseries('AAPL.O', start_date='2020-12-1',
                         end_date='2021-1-1', fields='CLOSE',
                         interval='daily')

In [ ]:
data.info()

## Index Constituents

In [ ]:
con, err = ek.get_data('0#.GDAXI', fields='CF_NAME')

In [ ]:
len(con)

In [ ]:
con.head()

## Time Series Data

In [ ]:
%%time
data = None
for ric in con['Instrument']:
    raw = ek.get_timeseries(ric, start_date='2016-1-1',
                        end_date='2021-1-1', fields='CLOSE',
                        interval='daily')
    if data is None:
        data = raw
        data.columns.name = None
        data.columns = [ric,]
    else:
        data[ric] = raw

In [ ]:
data = data[sorted(data.columns)]

In [ ]:
data.columns

In [ ]:
data.iloc[:5, :5]

In [ ]:
# data.to_csv('dax_eikon_eod_data.csv')

## Market Capitalization

In [ ]:
mc, err = ek.get_data('0#.GDAXI', fields=['CF_NAME', 'TR.CompanyMarketCap'])

In [ ]:
mc = mc.sort_values('Instrument')

In [ ]:
1e9

In [ ]:
mc['Company Market Cap'] = mc['Company Market Cap'] / 1e9

In [ ]:
mc.columns = ['RIC', 'NAME', 'MC']

In [ ]:
mc['MC%'] = mc['MC'].apply(lambda x: x / mc['MC'].sum())

In [ ]:
mc.head()

In [ ]:
mc.sort_values('MC%').set_index('RIC').plot.bar(figsize=(10, 6), subplots=True);

In [ ]:
mc.set_index('RIC')['MC%'].sort_values().plot.pie(cmap='coolwarm', figsize=(8, 8));

In [ ]:
mc = mc.set_index('RIC')
# mc

In [ ]:
# mc.to_csv('dax_eikon_mc_data.csv')  # 18. March 2021

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="30%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>